In [1]:
!pip install rouge --quiet
!pip install bert_score --quiet
!pip install openai --quiet

In [75]:
import openai
import os
import re
import pandas as pd

# Python Implementation of the ROUGE Metric
from rouge import Rouge

# BERTScore leverages the pre-trained contextual embeddings from BERT and matches words in candidate and reference sentences by cosine similarity.
from bert_score import BERTScorer

In [31]:
#Lettura Dataset
df = pd.read_json('release/test.jsonl', lines=True, orient='records')

In [32]:
#Selezione dei riassunti estrattivi tra quelli target. Il dataset ne contiene anche di astrattivi e misti 
df = df.loc[df["density_bin"] == 'extractive']
df.reset_index()

,index,url,archive,title,date,text,summary,compression,coverage,density,compression_bin,coverage_bin,density_bin
0,0,http://www.nydailynews.com/archives/news/1995/...,http://web.archive.org/web/20090428161725id_/h...,"SELENA'S LAST CRIES SHOT SINGER BEGGED HELP, N...",1970-08-21 12:40:28.161725,By MATT SCHWARTZ in Houston and WENDELL JAMIES...,"Bleeding from a massive chest wound, Tejano st...",6.965116,0.988372,25.406977,low,high,extractive
1,1,http://www.aol.com/2011/10/18/renee-nicole-dou...,http://web.archive.org/web/20111020094245id_/h...,Renee-Nicole Douceur Rescued: Sick South Pole ...,1970-08-21 18:23:40.094245,"By HOLLY RAMER, Associated Press\n\nCONCORD, N...","By HOLLY RAMER, Associated Press CONCORD, N.H....",6.883721,1.000000,32.674419,low,high,extractive
2,7,http://www.sfgate.com/bayarea/article/Thriving...,http://web.archive.org/web/20121121211221id_/h...,Thriving entrepreneurs of Mid-Market,1970-08-21 21:12:01.211221,Everyone is so excited about all the new busin...,Breathless stories have extolled the benefits ...,3.478548,0.914191,22.386139,low,medium,extractive
3,9,http://www.nytimes.com/1981/08/05/us/analyst-s...,http://web.archive.org/web/20130707162746id_/h...,ANALYST'S VIEWS ON NIJINSKY FINALLY PUBLISHED,1970-08-21 23:51:47.162746,"Vaslav Nijinsky, the great Russian dancer who ...","Vaslav Nijinsky, the great Russian dancer who ...",7.095960,0.984848,49.479798,low,high,extractive
4,12,http://www.theguardian.com/artanddesign/2008/j...,http://web.archive.org/web/20130902093632id_/h...,Rhoda Buchanan on political photographer Calli...,1970-08-21 23:55:02.093632,"She sees the holes in his shoes, his lucky cha...","Rhoda Buchanan: For 18 months, photographer C...",20.433333,0.900000,15.300000,medium,medium,extractive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36160,108855,http://www.washingtonpost.com/wp-dyn/content/a...,https://web.archive.org/web/2005052919id_/http...,Assault On the Media,1970-01-01 00:33:25.052919,So it turns out that the FBI has documents sho...,So it turns out that the FBI has documents sho...,26.294118,1.000000,34.000000,medium,high,extractive
36161,108856,http://www.washingtonpost.com/wp-dyn/content/a...,https://web.archive.org/web/2005052919id_/http...,"25 Years Later, It's Still 9:30",1970-01-01 00:33:25.052919,"ON MAY 30, 1980, the post-punk jazz Lounge Liz...","ON MAY 30, 1980, the post-punk jazz Lounge Liz...",21.195652,1.000000,46.000000,medium,high,extractive
36162,108857,http://www.washingtonpost.com/wp-dyn/content/a...,https://web.archive.org/web/2005052919id_/http...,Season Is a Success for Everyone Except NBC,1970-01-01 00:33:25.052919,"Ahumdinger TV season wrapped Wednesday night, ...","Ahumdinger TV season wrapped Wednesday night, ...",10.100000,0.983333,58.016667,low,high,extractive
36163,108858,http://www.washingtonpost.com/wp-dyn/content/a...,https://web.archive.org/web/2005052919id_/http...,Energy Package Clears Senate Committee,1970-01-01 00:33:25.052919,A Senate panel reached bipartisan agreement on...,A Senate panel reached bipartisan agreement on...,29.875000,1.000000,24.000000,medium,high,extractive


In [33]:
df.drop(['archive', 'title', 'date', 'compression', 'coverage', 'density', 'compression_bin', 'coverage_bin', 'density_bin'], inplace=True, axis=1)

In [34]:
#Selezione delle prime 10 recensioni estrattive
df10 = df[:10]

In [35]:
del df

In [ ]:
#Funzione di pulizia del testo
def text_cleaner(text):
    clean_text = re.sub(r'\n\n', ' ', text)
    clean_text = re.sub(r'\n', ' ', clean_text)
    clean_text = clean_text.replace('\\', '')
    return clean_text

In [40]:
from summarizer import Summarizer
import inspect  

model=Summarizer()

print(inspect.signature(model))  

(body: str, ratio: float = 0.2, min_length: int = 40, max_length: int = 600, use_first: bool = True, algorithm: str = 'kmeans', num_sentences: int = None, return_as_list: bool = False) -> str


In [43]:
#Predizione delle sintesi

from summarizer import Summarizer
results = []
model = Summarizer()
for i in range(1,11):
    text = df10['text'].iloc[i]
    clean_text = text_cleaner(text)
    clean_text_len = len(clean_text)
    target_summary = df10['summary'].iloc[i]
    #target_summary_len = len(target_summary.split())
    target_summary_len = len(target_summary)
    ratio = target_summary_len/clean_text_len
    #La lunghezza delle predizioni deve avere lo stesso ratio di compressione che ha la sintesi target
    #rispetto all'articolo originale
    result = model(clean_text, ratio=ratio)
    full = ''.join(result)
    results.append([full,target_summary])
    print("\nCOMPARISON N°:" + str(i) +"\n")
    print("Article:\n\n" + clean_text + "\n\nPredicted Summary:\n\n" + full + "\n\nTarget Summary:\n\n" + target_summary)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)



COMPARISON N°:0

Article:

By MATT SCHWARTZ in Houston and WENDELL JAMIESON in New York Daily News Writers Saturday, October 14th 1995, 4:22AM Bleeding from a massive chest wound, Tejano star Selena cried, "Help me! Help me! I've been shot!" and then named her killer with her dying breath. Shaken witnesses yesterday told a spellbound Houston courtroom how the blood-covered, mortally wounded 23-year-old Hispanic singing sensation burst into the lobby of the Corpus Christi Days Inn last March 31. Gasping for breath, Selena told motel workers that Yolanda Saldivar the president of her fan club shot her once in the back. She begged, "Close the door or she will shoot me again," the witnesses said. The testimony came on the third day of Saldivar's trial on charges she murdered Selena with a shot from a .38-caliber revolver when the star tried to fire her for embezzling $30,000 from two boutiques she managed for the singer. As a paramedic and motel workers recounted Selena's last desperate m

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)



COMPARISON N°:1

Article:

By HOLLY RAMER, Associated Press CONCORD, N.H. -- A sick American engineer who was successfully evacuated from the South Pole to New Zealand is awaiting the results of medical tests after having what doctors believed was a stroke in August. Renee-Nicole Douceur told The Associated Press in an email Tuesday that she had MRI and echocardiogram exams. She said results will be shared with doctors in the United States, "so everyone will be on the same page." "Back at hotel now to chill out," Douceur wrote. She added, "So nice to see green and smell freshly cut grass, flowers, birds chirping, insects, etc., since it's now been over a year on the flat polar plateau of just ice and snow." She landed in Christchurch on Monday. Douceur, 58, is a Seabrook, N.H., resident who worked as a manager for research station contractor Raytheon Polar Services Co. She asked for an emergency evacuation after having what doctors believed was a stroke in August, but officials reject

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


...

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)



COMPARISON N°:7

Article:

Now that Matthew Weiner has put the lie to “Leave It to Beaver” and “Make Room for Daddy,” revealing the denial, need, and aggression behind the perfect 1950s family façade, we wait. We wait — something that, in this age of “on demand” and TV binging, we rarely do — to find out exactly how the “Mad Men” creator will leave each of his characters when his drama ends next year. Will the symbolism-minded Weiner show these people rising to a higher floor, elevator-style; tumbling in free-fall like the cartoon man in the title sequence; or going up and down and round and round “on a carousel”? At the end of the first half of season seven last week, all signs pointed to a happy ending, to the tune of “The Best Things in Life Are Free.” Yes, “Mad Men” and a happy ending, a somewhat oxymoronic pairing. The show has been a long, slow slide into the darkest corners of its characters’ personalities: Betty’s childlike despair, Peggy’s loneliness, Roger’s nihilism, Joan’s

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)



COMPARISON N°:8

Article:

July 18, 2014: Audio released by Ukraine’s security agency appears to reveal separatists and Russian military officials acknowledging the shooting down of a civilian plane. Phone calls allegedly intercepted of pro-Russian rebels fighting in eastern Ukraine appear to confirm that the separatists shot down a commercial aircraft with 295 people onboard, including 27 Australians. Malaysia Airlines flight MH17 from Amsterdam to Kuala Lumpur was shot down over Ukraine's east overnight. Authorities do not expect any survivors. A video claiming to be a tapped phone conversation between pro-Russian rebel commanders appears to confirm that the Boeing 777 airplane was taken down deliberately. The authenticity of the video, which appears to have been released by Ukrainian intelligence services, is unknown. "We have just shot down a plane," a rebel commander identified as 'Bes' says. "It fell down outside Enakievo." July 18, 2014: Amateur vision has surfaced of the final

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [62]:
# function to calculate the Rouge score
def get_rouge_scores(text1, text2):
    rouge = Rouge()
    return rouge.get_scores(text1, text2)


rouge_scores_out = []

rouge_scores = []
for result in results:
    rouge_score = get_rouge_scores(result[0],result[1])
    rouge_scores.append(rouge_score)

for metric in ["rouge-1", "rouge-2", "rouge-l"]:
    for label in ["F-Score"]:
        summary_1 = rouge_scores[0][0][metric][label[0].lower()]
        summary_2 = rouge_scores[1][0][metric][label[0].lower()]
        summary_3 = rouge_scores[2][0][metric][label[0].lower()]
        summary_4 = rouge_scores[3][0][metric][label[0].lower()]
        summary_5 = rouge_scores[4][0][metric][label[0].lower()]
        summary_6 = rouge_scores[5][0][metric][label[0].lower()]
        summary_7 = rouge_scores[6][0][metric][label[0].lower()]
        summary_8 = rouge_scores[7][0][metric][label[0].lower()]
        summary_9 = rouge_scores[8][0][metric][label[0].lower()]
        summary_10 = rouge_scores[9][0][metric][label[0].lower()]

        row = {
            "Metric": f"{metric} ({label})",
            "Summary 1": summary_1,
            "Summary 2": summary_2,
            "Summary 3": summary_3,
            "Summary 4": summary_4,
            "Summary 5": summary_5,
            "Summary 6": summary_6,
            "Summary 7": summary_7,
            "Summary 8": summary_8,
            "Summary 9": summary_9,
            "Summary 10": summary_10,
            }
        rouge_scores_out.append(row)

'''
def highlight_max(s):
    is_max = s == s.max()
    return [
        "background-color: lightgreen" if v else "background-color: white"
        for v in is_max
    ]
'''

rouge_scores_out = (
    pd.DataFrame(rouge_scores_out)
    .set_index("Metric")
)

rouge_scores_out

,Summary 1,Summary 2,Summary 3,Summary 4,Summary 5,Summary 6,Summary 7,Summary 8,Summary 9,Summary 10
Metric,,,,,,,,,,
rouge-1 (F-Score),0.424658,0.772277,0.275229,0.51145,0.153846,0.044444,0.396947,0.170732,0.181818,0.520548
rouge-2 (F-Score),0.341463,0.740741,0.124197,0.40000,0.000000,0.000000,0.277778,0.020619,0.000000,0.432749
rouge-l (F-Score),0.424658,0.772277,0.262997,0.48855,0.115385,0.044444,0.381679,0.146341,0.181818,0.520548


* **ROUGE-1** misura l'overlap di parole.
* **ROUGE-2** è simile ma misura l'overlap di bigrammi ovvero parole adiacenti
* **ROUGE-L** cerca la più lunga sotto-sequenza di parole in comune

In [66]:
# Instantiate the BERTScorer object for English language
scorer = BERTScorer(lang="en")

for i in range(10):
    P, R, F1 = scorer.score([results[i][0]], [results[i][1]])
    print("Summary_" + str(i+1) + " F1 Score:", F1.tolist()[0])

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary_1 F1 Score: 0.8632969260215759
Summary_2 F1 Score: 0.9585409164428711
Summary_3 F1 Score: 0.8277502059936523
Summary_4 F1 Score: 0.8846156001091003
Summary_5 F1 Score: 0.8256981372833252
Summary_6 F1 Score: 0.8115198612213135
Summary_7 F1 Score: 0.8756425380706787
Summary_8 F1 Score: 0.8446834087371826
Summary_9 F1 Score: 0.8719710111618042
Summary_10 F1 Score: 0.9006274938583374


Paragonando gli embedding di entrambe le sintesi, BERTScore cattura le similarità semantiche che le metriche basate su n-grammi potrebbero perdere.